#### student Info
Scientific Caluculation & Machine Learning  
022-23 Semester 2, AssessmentTask 3  
Due time: CNT 23:59, FRI. 30 June 2023
Reference style: Harvard  
Subject lecturer: Prof. Hong-qian SANG   
Dept. : School of Artificial Intelligence     

# 基于神经网络的手写数字识别


## Introduction


### 问题介绍
手写字识别是一个数据分类问题。MNIST数据集提供的训练样本已经进行了人工标记，我们可以选择一种分类器进行监督学习，例如scikl-learn库中提供的线性分类器、支持向量机、K-最近邻、决策树、随机森林等，可以得到不错的模型准确率（>98%）。神经网络作为一种更为强大的工具，可以实现更高的分类准确率。

### 数据集的介绍
本作业的数据集为Prof.Sang指定的MNIST数据库

MNIST（Modified National Institute of Standards and Technology）是一个大型的手写数字数据库。该数据库还广泛用于机器学习领域的训练和测试。MNIST 数据库包含 60,000 张训练图像和 10,000 张测试图像，均为28像素乘28像素的灰度图。
#### 数据集的读取
由于没有使用高级库，所以使用Numpy直接读取存放在本地的mnist.npz文件。

f = np.load('mnist.npz')

X_train, y_train = f['x_train'], f['y_train']

X_test, y_test = f['x_test'], f['y_test'] 

f.close()

本作业在运行时将28像素乘28像素的灰度图样本转为长度为764的Numpy Array


## Assessment Task & Final

### 作业地址：
https://github.com/sim42/SCML/blob/main/Chapter15/AssessmentTask.ipynb

课程教师Prof.Sang提供了多层感知机（MLP）和卷积神经网络（CNN）的示例Python代码。基于这些示例代码，修改了并搭建一个自选结构的神经网络，并使用MNIST数据集训练一个手写数字识别模型。

### 要求与评分标准:

模型准确率要求大于98.5%。考试成绩取决于准确率的数值。
不允许使用Tensorflow、PyTorch等高级机器学习库，只可以使用Numpy等底层库手动实现特定算法。
### 提交格式规范
当前实例中将训练后的神经网络实例保存为.pkl文件，该文件要求和存有程序代码的.py文件

#### 准备：本作业的环境配置，包版本说明
NumPy:版本：1.17.4

Numba:版本：0.48.0

tqdm:版本：4.65.0

Matplotlib:版本：3.1.2

CPU型号：x86_64

Python版本：3.8.10 (default, May 26 2023, 14:05:08) [GCC 9.4.0]

操作系统：Linux

版本号：5.15.0-75-generic



## 获取环境配置，包版本说明

In [2]:

!pip show numpy numba tqdm matplotlib
import sys
import platform
os_name = platform.system()  # 获取操作系统名称
os_version = platform.release()  # 获取操作系统版本
processor = platform.processor()  # 获取CPU型号
processor = platform.processor()  # 获取CPU型号
print("CPU型号：", processor)
print("Python 版本:", sys.version)
print("操作系统：", os_name)
print("版本号：", os_version)




Name: numpy
Version: 1.17.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /usr/lib/python3/dist-packages
Requires: 
Required-by: sysmon, pyqtgraph, numba
---
Name: numba
Version: 0.48.0
Summary: compiling Python code using LLVM
Home-page: http://numba.github.com
Author: Anaconda, Inc.
Author-email: numba-users@continuum.io
License: BSD
Location: /home/jiangjing/.local/lib/python3.8/site-packages
Requires: llvmlite, numpy, setuptools
Required-by: 
---
Name: tqdm
Version: 4.65.0
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: None
Author-email: None
License: MPLv2.0, MIT Licences
Location: /home/jiangjing/.local/lib/python3.8/site-packages
Requires: 
Required-by: 
---
Name: matplotlib
Version: 3.1.2
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-ema

### 包的导入说明
numpy：用于进行科学计算和数值操作，提供了高效的数组和矩阵运算功能。

numba：用于加速Python代码的执行速度，通过即时编译技术将Python代码转换为快速的机器码。

tqdm：用于在循环中显示进度条，方便用户了解代码运行的进度。

pickle：用于序列化和反序列化Python对象，可以将对象保存到文件中或从文件中加载对象。

time：用于测量代码的执行时间，方便进行性能分析和优化。

matplotlib.pyplot：用于绘制图表和可视化数据

In [3]:
import numpy as np
import numba as nb
from tqdm import tqdm
import pickle
import time
import matplotlib.pyplot as plt

## 1.各种模块的定义

### 1.1激活函数   
在卷积神经网络中，激活函数是一种用于引入非线性变换的函数。它的作用是在每个神经元的输出上引入非线性关系，从而使神经网络能够学习和表示更加复杂和非线性的函数关系。  
为什么需要激活函数呢？假设我们使用线性变换（例如简单的乘法和加法）来构建神经网络，那么整个网络就只能表示线性函数。线性函数的表达能力有限，很难处理复杂的模式和关系。而激活函数的引入可以使神经网络具备更强大的表达能力，能够处理更复杂的数据和模式。    

在CNN中，常用的激活函数有以下几种：    
   
1.Sigmoid函数（S形函数）：
Sigmoid函数是最常见的激活函数之一。它将输入的值映射到一个介于0和1之间的范围内。在神经网络中，这个范围可以表示概率或者激活的程度。Sigmoid函数的公式是：f(x) = 1 / (1 + e^(-x))。然而，由于Sigmoid函数在输入非常大或非常小的时候会饱和，导致梯度消失的问题，因此在一些情况下并不推荐使用。  

2.ReLU函数（修正线性单元）：   
ReLU函数是近年来在CNN中广泛使用的激活函数。它将负值映射为零，而正值保持不变。ReLU函数的公式是：f(x) = max(0, x)。相比于Sigmoid函数，ReLU函数的计算速度更快，并且没有梯度消失的问题。这使得它成为了许多深度学习模型的首选激活函数。  

3.Leaky ReLU函数：    
Leaky ReLU函数是ReLU函数的一种扩展形式，它在负值部分引入了一个小的斜率。这样可以解决ReLU函数在负值部分可能出现的“神经元死亡”问题。Leaky ReLU函数的公式是：f(x) = max(0.01x, x)。通过引入非零斜率，Leaky ReLU函数在训练过程中可以更好地处理负值。   

这些是常见的CNN激活函数，每个函数都有自己的特点和适用场景，并且实际应用中，选择合适的激活函数会影响神经网络的性能和训练效果。 

### 1.2激活函数类说明：
这段代码定义了一个名为Activate的类 。

其中包含了两个静态方法apply_activation和apply_activation_derivative用于应用激活函数和计算激活函数的导数。   

apply_activation方法
它接受两个参数，r表示输入的值，activation表示所选择的激活函数。如果activation为None，则直接返回输入值r。如果activation为'relu'，则应用ReLU函数，即返回r和0的最大值。如果activation为'tanh'，则应用双曲正切函数，即返回np.tanh(r)。如果activation为'sigmoid'，则应用Sigmoid函数，即返回
1 / (1 + np.exp(-r))。  
如果activation为'softmax'，则应用Softmax函数，即对输入进行归一化处理，返回归一化后的结果。最后，如果activation不匹配任何已知的激活函数，则直接返回输入值r。

apply_activation_derivative方法：   
这个方法也接受两个参数，output表示激活函数的输出值，activation表示所选择的激活函数。如果activation为None，则返回一个与output大小相同的全为1的数组，表示导数为1。如果activation为'softmax'，则直接返回一个与output大小相同的全为1的数组，因为在softmax和交叉熵损失函数的情况下，导数为1。如果activation为'relu'，则根据output的值计算ReLU函数的导数，即大于0的值导数为1，小于等于0的值导数为0。  
如果activation为'tanh'，则返回1 - output ** 2。如果activation为'sigmoid'，则返回output * (1 - output)。要是activation不匹配任何已知的激活函数，则直接返回output。

In [4]:
class Activate:
    @staticmethod
    def apply_activation(r, activation=None):
        if activation is None:
            return r
        elif activation == 'relu':
            return np.maximum(r, 0)
        elif activation == 'tanh':
            return np.tanh(r)
        elif activation == 'sigmoid':
            #r = np.clip(r, -300, 300)  # clip the output to eliminate overflow error
            return 1 / (1 + np.exp(-r))
        elif activation == 'softmax':  # output layer
            r = r - np.max(r, axis=-1, keepdims=True)  # numerically stable softmax
            exp_r = np.exp(r)
            return exp_r / np.sum(exp_r, axis=-1, keepdims=True)
        return r
    @staticmethod
    def apply_activation_derivative(output, activation=None):
        """ Calculate the derivative of activation function """
        if activation is None: # No activation function, derivative is 1
            return np.ones_like(output)
        elif activation == 'softmax':  # Skip derivation when softmax and categorical cross-entropy loss are chosen
            return np.ones_like(output)
        elif activation == 'relu':
            grad = np.array(output, copy=True)
            grad[output > 0] = 1.
            grad[output <= 0] = 0.
            return grad
        elif activation == 'tanh':
            return 1 - output ** 2
        elif activation == 'sigmoid':
            return output * (1 - output)
        return output

### 1.2.卷积与卷积层
卷积层是CNN中最重要的组成部分之一。卷积层在图像处理和模式识别中起着非常重要的作用，它能够自动提取图像中的特征。     

卷积层的核心概念：  
卷积运算是卷积层的核心运算，卷积运算可以理解为将一个滤波器（也称为卷积核）应用于输入图像的不同位置，以产生输出特征图。这个滤波器是一个小的矩阵，它通过滑动窗口的方式在输入图像上移动，并与图像的每个局部区域进行逐元素相乘并求和。这个过程可以帮助我们检测图像中的边缘、纹理、颜色等特征。  


  
  
卷积层的工作原理如下： 

输入图像被表示为一个多维数组，比如一个RGB图像可以表示为一个三维数组（高度、宽度和通道数）。  
卷积层包含多个滤波器（卷积核），每个滤波器都学习不同的特征。每个滤波器都是一个小的矩阵，它通过滑动窗口的方式在输入图像上移动，并与图像的每个局部区域进行卷积运算。  

卷积运算的结果是一个特征图，它表示了滤波器在输入图像上检测到的特征的强度。每个特征图都对应着一个滤波器，可以理解为该滤波器对应的特征在输入图像中的分布情况。   

通过使用不同的滤波器，卷积层可以同时检测多个不同的特征，从而提取出更加丰富和复杂的图像特征。
卷积层的优势在于它具有局部连接和权值共享的特性。局部连接意味着每个神经元只与输入图像的一小部分区域连接，这样可以减少网络的参数数量。权值共享意味着每个滤波器在整个输入图像上都使用相同的权重，这样可以提取相似的特征并减少计算量。

卷积层是CNN中的核心组件之一，它的引入使得神经网络能够自动学习图像中的特征，从而提高了图像分类、目标检测和图像生成等任务的性能。


### 1.3卷积层定义的函数与各种类 
这段代码定义了一个卷积层类(ConvLayer)，它用于神经网络中的卷积操作。   

__init__(self, n_filters, kernel_size, activation=None, weights=None, bias=None, use_bias=True)：    
这是类的构造函数，用于初始化卷积层的参数。n_filters表示卷积核的数量，kernel_size表示卷积核的大小，activation表示激活函数，weights表示卷积核的权重，bias表示偏置项，use_bias表示是否使用偏置项。   


initialize(self)：   
这是一个初始化函数，用于对卷积核的权重和偏置项进行初始化。如果权重或偏置项为空，则会生成随机的初始值。这里使用了Xavier初始化方法。   

activate_jit(image, weights, bias, use_bias)：    
这是一个静态方法，使用了nb.jit装饰器进行加速。它实现了卷积操作，并返回卷积结果。   

activate(self, image)：  
这是前向传播函数，用于对输入图像进行卷积操作并经过激活函数处理后返回输出。   

back_propagation_jit(image, weights, bias, delta, use_bias)：    
这是一个静态方法，使用了nb.jit装饰器进行加速。它实现了反向传播过程，计算误差输入、权重梯度和偏置项梯度。    

back_propagation(self, image, error, output)：    
这是反向传播函数，用于计算误差输入，并更新权重梯度和偏置项梯度。  


In [5]:
class ConvLayer:
    def __init__(self, n_filters, kernel_size, activation=None, weights=None, bias=None, use_bias=True):
        self.n_filters = n_filters  # n_output_image = n_input_image * n_filters
        self.kernel_size = kernel_size
        self.weights = weights  # filters
        self.bias = bias
        self.activation = activation
        self.use_bias = use_bias
    def initialize(self):  # Xavier initialization
        if self.weights is None:
            self.weights = np.random.randn(self.n_filters, self.kernel_size[0], self.kernel_size[1]) / \
                            np.sqrt(self.kernel_size[0] * self.kernel_size[1])
        if self.bias is None:
            self.bias = np.zeros(self.n_filters)
        self.weights_gradient = np.zeros_like(self.weights)
        self.bias_gradient = np.zeros_like(self.bias)
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)  # accelerate C-style for-loop
    def activate_jit(image, weights, bias, use_bias):
        n_input_image, h, w = image.shape
        n_filters, h_kernel, w_kernel = weights.shape
        r = np.zeros((n_input_image * n_filters, h - h_kernel + 1, w - w_kernel + 1))
        for n in range(n_input_image):
            for i in range(h - h_kernel + 1):
                for j in range(w - w_kernel + 1):
                    for f in range(n_filters):
                        for i_kernel in range(h_kernel):
                            for j_kernel in range(w_kernel):
                                r[n * n_filters + f, i, j] += image[n, i + i_kernel, j + j_kernel] * weights[f, i_kernel, j_kernel]
                        if use_bias:
                            r[n * n_filters + f, i, j] += bias[f]
        return r
    def activate(self, image):
        """ Forward propagation """
        r = self.activate_jit(image, self.weights, self.bias, self.use_bias)
        output = Activate.apply_activation(r, self.activation)
        return output
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)
    def back_propagation_jit(image, weights, bias, delta, use_bias):
        n_input_image, h, w = image.shape
        n_filters, h_kernel, w_kernel = weights.shape
        weights_gradient = np.zeros((n_filters, h_kernel, w_kernel))
        bias_gradient = np.zeros(n_filters)
        error_input = np.zeros((n_input_image, h, w))
        for n in range(n_input_image):
            for i in range(h - h_kernel + 1):
                for j in range(w - w_kernel + 1):
                    for f in range(n_filters):
                        for i_kernel in range(h_kernel):
                            for j_kernel in range(w_kernel):
                                weights_gradient[f, i_kernel, j_kernel] += image[n, i + i_kernel, j + j_kernel] * delta[n * f + f, i, j]
                                error_input[n, i + i_kernel, j + j_kernel] += weights[f, i_kernel, j_kernel] * delta[n * f + f, i, j]
                        if use_bias:
                            bias_gradient[f] += delta[n * f + f, i, j]
        return error_input, weights_gradient, bias_gradient
    def back_propagation(self, image, error, output):
        assert error.shape == output.shape
        delta = error * Activate.apply_activation_derivative(output, self.activation)
        error_input, weights_gradient, bias_gradient = self.back_propagation_jit(image, self.weights, self.bias, delta, self.use_bias)
        self.weights_gradient += weights_gradient
        if self.use_bias:
            self.bias_gradient += bias_gradient
        return error_input

### 1.4池化与最大池化层
当我们处理图像时，需要用到池化？池化层是CNN中的一种重要层级，它有助于减少图像的维度，并提取出最重要的特征。   

在CNN中，卷积层负责检测图像中的特征，例如边缘、纹理和形状。然而，卷积层输出的特征图可能非常大，这会导致计算量过大和过拟合的问题。为了解决这个问题，我们引入了池化层。   

池化层通过减少特征图的尺寸来降低计算量，并保留最重要的特征。最常见的池化操作是最大池化（Max Pooling）。最大池化将输入图像划分为不重叠的矩形区域，并在每个区域中选择最大值作为输出。这样做的好处是，最大池化可以保留图像中最显著的特征，例如边缘和纹理。   

举个例子，假设我们有一个2×2的最大池化层，输入是一个4×4的特征图。我们将输入特征图划分为4个2×2的区域，并在每个区域中选择最大值作为输出。这样，我们的输出特征图将变为2×2，尺寸减小了一半。   

最大池化的另一个优点是它具有平移不变性。这意味着，无论特征出现在图像的哪个位置，最大池化都能够检测到它。这种平移不变性使得CNN在处理图像时更加鲁棒和有效。    

综上所述，池化层是CNN中的一种重要层级，通过减少特征图的尺寸和保留最重要的特征来降低计算量。最大池化是最常见的池化操作，它选择每个区域中的最大值作为输出。最大池化具有平移不变性，使得CNN在图像处理中更加鲁棒和有效。

### 1.5最大池化层的类与函数定义说明

定义MaxPoolingLayer的类，它接受一个参数pool_size，表示池化窗口的大小。  

类中定义了一个静态方法activate_jit，用于执行最大池化操作。它接受两个参数：image表示输入的图像数据，pool_size表示池化窗口的大小。该方法使用了Numba库进行加速，通过嵌套的循环实现了最大池化的计算过程。最终返回池化后的输出。

activate_jit(image, pool_size)：  
它接受一个参数image，调用了activate_jit方法来执行最大池化操作，并返回池化后的输出。   

activate(self, image):  
用于执行最大池化层的反向传播操作。它接受三个参数：image表示输入的图像数据，pool_size表示池化窗口的大小，error表示反向传播过程中的误差。该方法同样使用了Numba库进行加速，通过嵌套的循环实现了最大池化层反向传播的计算过程。最终返回误差在输入上的梯度。    

back_propagation_jit(image, pool_size, error):    
它接受三个参数：image表示输入的图像数据，error表示反向传播过程中的误差，output表示前向传播的输出。该方法调用了back_propagation_jit方法来执行反向传播操作，并返回误差在输入上的梯度。

这个类实现了最大池化层的前向传播和反向传播操作，并使用Numba库进行加速，能够提高计算效率。

In [6]:

class MaxPoolingLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)  # accelerate C-style for-loop
    def activate_jit(image, pool_size):
        n_input_image, h, w = image.shape
        h_pool = h // pool_size[0]
        w_pool = w // pool_size[1]
        output = np.zeros((n_input_image, h_pool, w_pool))
        for n in range(n_input_image):
            for i in range(h_pool):
                for j in range(w_pool):
                    max_pool = image[n, i*pool_size[0], j*pool_size[1]]
                    for i_pool in range(pool_size[0]):
                        for j_pool in range(pool_size[1]):
                            if max_pool < image[n, i*pool_size[0]+i_pool, j*pool_size[1]+j_pool]:
                                max_pool = image[n, i*pool_size[0]+i_pool, j*pool_size[1]+j_pool]
                    output[n,i,j] = max_pool
        return output
    def activate(self, image):
        output = self.activate_jit(image, self.pool_size)
        return output
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)
    def back_propagation_jit(image, pool_size, error):
        n_input_image, h, w = image.shape
        h_pool = h // pool_size[0]
        w_pool = w // pool_size[1]
        error_input = np.zeros((n_input_image, h, w))
        for n in range(n_input_image):
            for i in range(h_pool):
                for j in range(w_pool):
                    max_pool = image[n, i*pool_size[0], j*pool_size[1]]
                    max_i, max_j = (0, 0)
                    for i_pool in range(pool_size[0]):
                        for j_pool in range(pool_size[1]):
                            if max_pool < image[n, i*pool_size[0]+i_pool, j*pool_size[1]+j_pool]:
                                max_pool = image[n, i*pool_size[0]+i_pool, j*pool_size[1]+j_pool]
                                max_i = i_pool
                                max_j = j_pool
                    error_input[n, i*pool_size[0] + i_pool, j*pool_size[1] + j_pool] = error[n, i, j]
        return error_input
    def back_propagation(self, image, error, output):
        assert error.shape == output.shape
        error_input = self.back_propagation_jit(image, self.pool_size, error)
        return error_input

### 1.6特征工程与扁平化层

特征工程是指在机器学习或数据挖掘任务中，根据对问题的理解和领域知识，对原始数据进行转换和处理，以提取出更有意义、更能表达问题的特征。特征工程的目的是为了改善模型的性能，提高预测或分类的准确性。

这是特征工程中常见的一些操作和技术：

1特征选择（Feature Selection）：从原始数据中选择与目标变量相关性较高的特征，以减少特征的维度和冗余信息。常见的方法包括相关性分析、方差选择、递归特征消除等。

2特征提取（Feature Extraction）：将原始数据转换为更有意义的表示形式，以便于模型理解和处理。常见的方法包括主成分分析（PCA）、线性判别分析（LDA）、非负矩阵分解（NMF）等。

3特征构造（Feature Construction）：通过对原始特征进行组合、变换或衍生，创建新的特征来捕捉数据中的更多信息。常见的方法包括多项式特征、交互特征、时间序列特征等。

4缺失值处理（Missing Value Handling）：对于含有缺失值的特征，常见的处理方法包括删除缺失值、用平均值或中位数填充缺失值、使用模型预测缺失值等。

5标准化（Normalization）：将特征的值按照一定的规则进行缩放，以确保各个特征具有相似的尺度。常见的方法包括Z-score标准化、最大最小值标准化等。

6独热编码（One-Hot Encoding）：将分类变量转换为二进制编码形式，以便于模型处理。例如，将颜色特征的取值（红、绿、蓝）转换为三个二进制特征。

7特征重要性评估（Feature Importance Evaluation）：通过模型训练的结果，评估每个特征对预测结果的重要性，以便进一步筛选和调整特征。

而在CNN中，扁平化层（Flatten layer）是将卷积层和池化层输出的多维特征图转换为一维向量的操作。这样做的目的是为了将特征图的空间结构转换为线性结构，以便输入到全连接层进行进一步的处理和分类。

作为机器学习的初学者，可以通过一个简单的比喻来解释特征工程和扁平化层的概念。假设你要画一只猫的轮廓，你可以通过观察猫的眼睛、鼻子、嘴巴等特征，将这些特征组合起来画出猫的轮廓。这就好比特征工程，通过观察和选择图像中的特征来提取有用的信息。而扁平化层就好比将画好的轮廓展开成一条直线，方便进行进一步的处理和分类。

希望这个比喻能帮助你更好地理解特征工程和扁平化层的概念。如果还有其他问题，我会尽力帮助你解答。

### 1.7扁平化层的类与函数定义说明
这块代码定义了一个名为FlattenLayer的类，它是一个扁平化层。扁平化层将输入的多维特征图转换为一维向量。  

在类的构造函数__init__中，它初始化了一个成员变量image_shape，用于保存输入图像的形状。  

def activate(self, image):  
它接受一个参数image，并将image_shape设置为输入图像的形状。然后，通过调用image.flatten()方法将输入图像扁平化为一维向量，并返回扁平化后的输出。  

def back_propagation(self, image, error, output):  
它接受三个参数：image表示输入的图像数据，error表示反向传播过程中的误差，output表示前向传播的输出。该方法首先通过调用error.reshape(self.image_shape)方法将误差恢复为与输入图像相同的形状。然后，将恢复后的误差返回作为误差在输入上的梯度。    

宗上所叙述，这个类实现了扁平化层的前向传播和反向传播操作，用于将输入的多维特征图转换为一维向量，并在反向传播时将误差恢复为与输入图像相同的形状。

In [7]:

class FlattenLayer:
    def __init__(self):
        self.image_shape = None
    def activate(self, image):
        self.image_shape = image.shape
        output = image.flatten()
        return output
    def back_propagation(self, image, error, output):
        assert error.shape == output.shape
        error_input = error.reshape(self.image_shape)
        return error_input

### 1.8神经网络与全连接层

神经网络是一种受到生物神经系统启发的计算模型，用于学习和解决各种问题。它由一系列相互连接的神经元组成，这些神经元模拟了生物神经元之间的连接和信息传递。  

神经网络通常由多个层组成，每一层由多个神经元组成。最常见的神经网络是前馈神经网络，其中信息从输入层经过隐藏层传递到输出层，每个神经元都与前一层的神经元连接。这种结构使得神经网络可以学习输入和输出之间的复杂关系。  

每个神经元接收来自前一层神经元的输入，并对这些输入进行加权求和，然后通过一个非线性的激活函数进行处理。 通过调整权重，神经网络可以学习到最优的表示和预测模式。训练神经网络的过程涉及两个主要步骤：前向传播和反向传播。  

在前向传播过程中，输入数据通过网络的各个层传递，每个神经元根据权重和激活函数计算输出。输出层的结果与真实值进行比较，得到误差。然后，在反向传播过程中，误差通过网络反向传递，根据误差调整权重，以减小误差。这个过程不断重复，直到网络的预测结果达到期望的精度。


在CNN中，全连接层（Fully Connected Layer）就算是神经网络的一部分，也被称为密集连接层或输出层。全连接层位于神经网络的最后一层，它的每个神经元都与前一层的所有神经元相连接。

假设我们正在处理一个图像分类任务，该任务需要将图像分为不同的类别。在经过卷积层和池化层等操作之后，我们得到了一个具有高级特征的输出。全连接层的作用是将这些高级特征与不同的类别进行关联，从而得出最终的分类结果。

全连接层的每个神经元都有一个权重向量，它与前一层的神经元的输出进行加权求和，然后通过一个激活函数进行处理。这样，全连接层可以学习到输入数据的复杂关系，并输出对不同类别的预测结果。

现在本作业的任务是对手写数字进行分类，有10个类别（0到9）。在CNN中，全连接层通常有10个神经元，每个神经元对应一个数字类别。通过训练，神经网络会学习到每个神经元与不同数字之间的关联，从而能够对输入的手写数字进行准确的分类。

全连接层的作用是将高级特征与类别之间建立起联系，使得神经网络可以进行分类和预测。它是神经网络中非常重要的一部分，也是本图像分类作业代码的关键组成部分。

### 1.9全连接层的定义的类，函数定义说明
这块码定义了一个名为DenseLayer的类，表示一个全连接层。

__init__(self, n_output, activation=None, weights=None, bias=None, use_bias=True)：初始化方法，用于创建DenseLayer对象。参数包括：  

n_output：表示该层的输出维度（神经元个数）。  
activation：表示该层的激活函数，可以是relu、tanh、sigmoid或softmax。  
weights：表示该层的权重，可以是预先定义的权重矩阵。  
bias：表示该层的偏置，可以是预先定义的偏置向量。  
use_bias：表示是否使用偏置，默认为True。 

initialize(self, n_input)：  
初始化方法，用于初始化该层的权重和偏置。参数n_input表示该层的输入维度（神经元个数）。  
如果weights为None，则随机生成一个形状为(n_input, n_output)的权重矩阵，并使用Xavier初始化方法进行初始化。  
如果bias为None，则随机生成一个形状为(n_output,)的偏置向量，并将其元素初始化为0.1的倍数。  
初始化权重梯度和偏置梯度为与权重和偏置相同形状的零矩阵和零向量。  
activate(self, X)：前向传播方法，用于计算该层的输出。参数X为输入向量。  
首先将输入向量与权重矩阵相乘，得到X@W。   
如果use_bias为True，则将偏置向量加到X@W上。   
将得到的结果应用激活函数，得到输出向量。  

back_propagation(self, X, error, output)：  
反向传播方法，用于计算该层的梯度并返回误差输入。参数包括：   
X：输入向量。  
error：输出误差。  
output：该层的输出向量。  
首先计算输出误差的导数乘以激活函数导数，得到delta。  
计算误差输入，即上一层的输出误差，通过将权重矩阵与delta相乘得到。  
更新权重梯度和偏置梯度，分别为delta * X[np.newaxis].T和delta。返回误差输入。  
该类定义了一个全连接层的基本功能，包括初始化权重和偏置、前向传播计算输出、反向传播计算梯度。

In [8]:

class DenseLayer:
    """ A Fully Connected Layer """
    def __init__(self, n_output, activation=None, weights=None, bias=None, use_bias=True):
        self.n_output = n_output
        self.weights = weights
        self.bias = bias
        self.activation = activation  # relu tanh sigmoid or softmax
        self.use_bias = use_bias
    def initialize(self, n_input):  # Xavier initialization
        self.n_input = n_input
        if self.weights is None:
            self.weights = np.random.randn(self.n_input, self.n_output) / np.sqrt(self.n_input)
        if self.bias is None:
            self.bias = np.random.rand(self.n_output) * 0.1
        self.weights_gradient = np.zeros_like(self.weights)
        self.bias_gradient = np.zeros_like(self.bias)
    def activate(self, X):
        """ Forward propagation """
        assert X.ndim == 1
        r = np.dot(X, self.weights)  # X@W + b
        if self.use_bias:
            r += self.bias
        output = Activate.apply_activation(r, self.activation)
        return output
    def back_propagation(self, X, error, output):
        assert error.shape == output.shape
        delta = error*Activate.apply_activation_derivative(output, self.activation)  # calculate delta of X@W + b
        error_input = np.dot(self.weights, delta)  # Chain Rule
        self.weights_gradient += delta * X[np.newaxis].T
        if self.use_bias:
            self.bias_gradient += delta
        return error_input

### 1.10过拟合怎么办？加入正则化与随机失活层

当我们训练神经网络时，我们经常会遇到过拟合的问题。过拟合指的是模型在训练集上表现良好，但在新数据上表现较差，类比：学生复习的时候死记硬背，可以把习题集全做对，而考试一塌糊涂。为了解决这个问题，我们可以使用正则化和随机失活层。  

正则化是一种常用的技术，它通过在损失函数中引入额外的惩罚项来限制模型的复杂度。这个惩罚项可以是L1正则化或L2正则化。L1正则化通过加上权重的绝对值之和来惩罚模型中的大权重，从而使模型更倾向于选择较少的特征。L2正则化则通过加上权重的平方和来惩罚模型中的大权重，从而使模型更倾向于选择权重较小的特征。正则化可以帮助我们在训练过程中更好地平衡模型的拟合能力和泛化能力，从而减少过拟合的风险。  

随机失活也可以解决过拟合，它在训练过程中随机地将一部分神经元的输出设置为0，从而使它们在计算中被忽略。这样做的目的是强制模型不依赖于特定的神经元，从而增强其泛化能力。随机失活可以看作是对不同子模型的集成，每个子模型都是通过将某些神经元的输出设为0而得到的。在测试时，我们通常会对所有的神经元进行计算，但是需要将输出值按照训练时失活的比例进行缩放，以保持一致性。

通过正则化和随机失活层，我们可以有效地减少过拟合的风险，提高模型的泛化能力。

### 1.11随机失活层的类，函数定义说明
这部分的代码定义了一个名为DropoutLayer的类，它是一个随机失活层，用于减少过拟合。  

类的构造函数__init__：  
它接受一个可选参数keep_prob，默认为1，表示每个神经元被保留的概率。  
keep_prob，用于保存keep_prob的值。另外，还定义了一个实例变量sample，用于保存随机生成的二进制掩码。    

实例方法activate(self, X):   
它接受一个参数X，X表示输入的向量。首先，通过assert语句检查X的维度是否为1，因为dropout通常被放置在全连接层或扁平化层之后。然后，使用np.random.binomial函数生成与X形状相同的二进制掩码sample，其中1的概率为keep_prob，0的概率为1-keep_prob。最后，将输入向量X与二进制掩码sample相乘，以实现随机失活。最后，将结果除以keep_prob，以保持期望值不变，并返回结果作为输出。  

back_propagation(self, X, error, output):  
它接受三个参数：X表示输入的向量，error表示反向传播过程中的误差，output表示前向传播的输出。该方法首先通过将误差乘以sample，将误差传播到未被失活的神经元上。然后，将得到的误差返回作为误差在输入上的梯度。

这个类实现了随机失活层的前向传播和反向传播操作，用于在训练过程中以一定概率随机失活神经元，从而减少过拟合。

In [9]:

class DropoutLayer:
    def __init__(self, keep_prob=1):
        self.keep_prob = keep_prob
        self.sample = None
    def activate(self, X):  # apply different binary mask to each example in a minibatch
        assert X.ndim == 1  # dropout is placed after dense layer or flatten layer
        self.sample = np.random.binomial(1, self.keep_prob, X.shape)
        X = X * self.sample
        output = X / self.keep_prob
        return output
    def back_propagation(self, X, error, output):
        assert error.shape == output.shape
        error_input = error * self.sample
        return error_input

### 1.12优化方法：从矩阵范数到随机剃度下降再到Adam

#### 1.12.1矩阵范数

矩阵范数是一种将矩阵映射到实数的函数，用于衡量矩阵的大小和变化程度。在数学中，常见的矩阵范数有Frobenius范数、1-范数、2-范数等。这些范数在最小二乘法中起着重要的作用。  


\begin{array}{c}
Frobenius范数（也称为2-范数）：[|A|F = \sqrt{\sum{i=1}^{m} \sum_{j=1}^{n} |a_{ij}|^2}]  
\end{array}

\begin{array}{c}
1-范数（也称为列和范数）：[|A|1 = \max{1\leq j \leq n} \sum_{i=1}^{m} |a_{ij}|]
\end{array}

\begin{array}{c}

2-范数（也称为谱范数或矩阵的最大奇异值）：[|A|2 = \sqrt{\lambda{\text{max}}(A^*A)}
\end{array}

  


#### 1.12.2最小二乘法与矩阵范数

最小二乘法是一种用于求解线性方程组的方法，尤其适用于超定方程组（方程个数大于未知数个数）的情况。最小二乘法的目标是找到一个与给定方程组的解最接近的解。在最小二乘法中，我们通常使用矩阵范数来衡量残差矩阵的大小，从而最小化误差。  

对于线性方程组Ax=b，其中A是一个m×n的矩阵，x是一个n维向量，b是一个m维向量，最小二乘法的目标是找到一个x，使得||Ax-b||最小。这里的||.||表示矩阵范数，可以是Frobenius范数、1-范数、2-范数等。    

不同的矩阵范数在最小二乘法中有不同的应用。例如，Frobenius范数是一种常用的矩阵范数，它可以衡量矩阵的总体大小。在最小二乘法中，我们可以使用Frobenius范数来衡量残差矩阵Ax-b的大小，从而最小化误差。1-范数和2-范数也常用于最小二乘法中，它们可以衡量矩阵的列向量和特征值的大小，从而对残差矩阵进行建模和求解。


#### 1.12.3矩阵范数与最优化
在优化问题中，我们通常会使用矩阵范数来定义目标函数或约束条件，并使用优化算法来求解最优解。特别是SGD和ADAM算法。这两个算法都是用于求解优化问题的常见算法。  

SGD是一种基本的优化算法，用于最小化目标函数。它的核心思想是通过迭代更新参数来逐步接近最优解。SGD在每次迭代中随机选择一小部分样本来计算梯度，并使用梯度的负方向来更新参数。这样做的好处是可以加快收敛速度，并且可以处理大规模数据集。然而，SGD的缺点是对于具有不同尺度的特征或参数，收敛速度可能会受到影响。

为了解决SGD的缺点，ADAM算法被提出。ADAM是一种自适应学习率的优化算法，它可以根据梯度的变化自动调整学习率。具体来说，ADAM使用动量和二阶矩估计来调整学习率。动量可以帮助算法在参数更新时保持稳定的方向，而二阶矩估计可以帮助算法调整学习率的大小。通过自适应地调整学习率，ADAM可以更好地适应不同尺度的特征或参数，从而提高收敛速度和性能。

#### 1.12.4二者联系
在SGD和ADAM算法中，矩阵范数可以用来定义目标函数或约束条件中的矩阵的大小。例如，在SGD算法中，我们可以使用矩阵范数来衡量梯度的大小，并根据梯度的大小来调整学习率。类似地，在ADAM算法中，我们可以使用矩阵范数来衡量动量和二阶矩估计的大小，并根据它们的大小来调整学习率的大小。

综上所述起来，SGD和ADAM算法则是常用的优化算法，它们利用矩阵范数来调整学习率，以加快收敛速度和提高性能。

### 1.13优化器的类，函数定义说明
这快代码定义Optimizers的类，该类包含了一些优化器方法和参数。

__init__方法：  
类的构造函数，用于初始化类的实例。  
它接受一些参数：  
optimizer（优化器类型）    
learning_rate（学习率）  
clipvalue（梯度裁剪值）  
decay_rate（学习率衰减率）  
decay_steps（学习率衰减步数），momentum（动量）  
nesterov（是否使用Nesterov动量）  
beta_1（ADAM优化器的参数）  
beta_2（ADAM优化器的参数）  
epsilon（ADAM优化器的参数）  

def initialize(self, layers, batch_size):    
这个方法用于初始化优化器的状态。它接受两个参数，layers（网络层列表）和batch_size（批量大小）。在该方法中，根据优化器类型对网络层的权重和偏置进行初始化。    
  
def apply(self, layers):     
这个方法用于应用优化器来更新网络层的权重和偏置。它接受一个参数layers（网络层列表）。在该方法中，对每个网络层的权重和偏置进行梯度平均化、梯度裁剪等操作，并根据优化器类型进行权重和偏置的更新。      

def sgd_jit(weights, gradient, velocity, momentum, learning_rate, nesterov):    
sgd_jit方法是一个静态方法，用于实现随机梯度下降（SGD）优化器。它接受一些参数，包括权重、梯度、速度、动量、学习率和Nesterov参数。在该方法中，根据动量和Nesterov参数更新权重，并返回更新后的权重和速度。  

def adam_jit(weights, gradient, first_moment, second_moment, beta_1, beta_2, epsilon, n_step, learning_rate):  
adam_jit方法是一个静态方法，用于实现自适应矩估计（ADAM）优化器。它接受一些参数，包括权重、梯度、一阶矩、二阶矩、beta_1、beta_2、epsilon、步数和学习率。在该方法中， 据ADAM优化器的公式更新权重，并返回更新后的权重、一阶矩和二阶矩。    

这些方法和参数用于实现不同的优化算法，例如随机梯度下降（SGD）和自适应矩估计（ADAM）。通过调用initialize方法初始化优化器的状态，然后通过调用apply方法来应用优化器来更新网络层的权重和偏置。

In [10]:


class Optimizers:
    def __init__(self, optimizer, learning_rate=0.001, clipvalue=None, decay_rate=0.99, decay_steps=1e4,
                 momentum=0.8, nesterov=True, beta_1=0.9, beta_2=0.999, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.clipvalue = clipvalue
        self.decay_rate, self.decay_steps = decay_rate, decay_steps  # exponential decay of learning rate
        self.momentum, self.nesterov = momentum, nesterov  # SGD
        self.beta_1, self.beta_2, self.epsilon = beta_1, beta_2, epsilon  # ADAM
    def initialize(self, layers, batch_size):
        self.batch_size = batch_size
        #self.learning_rate *= np.sqrt(batch_size)  # scale lr to keep the variance in the gradient expectation constant
        self.n_step = 1
        self.decay_steps = self.decay_steps // batch_size
        if self.optimizer == 'sgd':
            for layer in layers:
                if type(layer) == ConvLayer or type(layer) == DenseLayer:
                    layer.weights_velocity = np.zeros_like(layer.weights)
                    layer.bias_velocity = np.zeros_like(layer.bias)
        elif self.optimizer == 'adam':
            for layer in layers:
                if type(layer) == ConvLayer or type(layer) == DenseLayer:
                    layer.weights_first_moment = np.zeros_like(layer.weights)
                    layer.weights_second_moment = np.zeros_like(layer.weights)
                    layer.bias_first_moment = np.zeros_like(layer.bias)
                    layer.bias_second_moment = np.zeros_like(layer.bias)
        else:
            raise ValueError("Unknown Optimizer")
    def apply(self, layers):
        for layer in layers:
            if type(layer) == ConvLayer or type(layer) == DenseLayer:
                layer.weights_gradient /= self.batch_size  # averaging gradients over batch
                layer.bias_gradient /= self.batch_size
                if self.clipvalue is not None:  # avoid exploding gradients with gradient clipping
                    layer.weights_gradient = np.clip(layer.weights_gradient, -self.clipvalue, self.clipvalue)
                    layer.bias_gradient = np.clip(layer.bias_gradient, -self.clipvalue, self.clipvalue)
                if self.optimizer == 'sgd':
                    layer.weights, layer.weights_velocity = self.sgd_jit(layer.weights, layer.weights_gradient, layer.weights_velocity, 
                                                                         self.momentum, self.learning_rate, self.nesterov)
                    layer.bias, layer.bias_velocity = self.sgd_jit(layer.bias, layer.bias_gradient, layer.bias_velocity, 
                                                                   self.momentum, self.learning_rate, self.nesterov)
                elif self.optimizer == 'adam':
                    layer.weights, layer.weights_first_moment, layer.weights_second_moment = self.adam_jit(layer.weights, layer.weights_gradient, layer.weights_first_moment, layer.weights_second_moment, 
                                                                                                           self.beta_1, self.beta_2, self.epsilon, self.n_step, self.learning_rate)
                    layer.bias, layer.bias_first_moment, layer.bias_second_moment = self.adam_jit(layer.bias, layer.bias_gradient, layer.bias_first_moment, layer.bias_second_moment, 
                                                                                                  self.beta_1, self.beta_2, self.epsilon, self.n_step, self.learning_rate)
                layer.weights_gradient = np.zeros_like(layer.weights)
                layer.bias_gradient = np.zeros_like(layer.bias)
        if self.n_step % self.decay_steps == 0:
            self.learning_rate *= self.decay_rate
        self.n_step += 1
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)
    def sgd_jit(weights, gradient, velocity, momentum, learning_rate, nesterov):
        """ Stochastic dradient Descent (with momentum) """
        velocity = momentum * velocity + gradient
        if nesterov:
            weights -= (momentum * velocity + gradient) * learning_rate
        else:
            weights -= velocity * learning_rate
        return weights, velocity
    @staticmethod
    @nb.jit(nopython=True, fastmath=True)
    def adam_jit(weights, gradient, first_moment, second_moment, beta_1, beta_2, epsilon, n_step, learning_rate):
        """ Adaptive moment estimation """
        first_moment = beta_1 * first_moment + (1-beta_1) * gradient
        second_moment = beta_2 * second_moment + (1-beta_2) * gradient * gradient
        first_unbias = first_moment / (1-beta_1**n_step)
        second_unbias = second_moment / (1-beta_2**n_step)
        weights -= first_unbias / (np.sqrt(second_unbias) + epsilon) * learning_rate
        return weights, first_moment, second_moment

### 1.14神经网络的类与函数定义
定义了一个名为NeuralNetwork的类，包含了一系列方法和属性。

__init__(self)方法初始化类的属性。初始化了以下属性：  
 
_layers：存储神经网络的层的列表  
_mses：训练集上的均方误差（Mean Square Errors）  
_ces：训练集上的交叉熵（Cross Entropy）    
_accuracy：测试集上的准确率    

add_layer(self, layer)方法：  
用于向神经网络中添加层。    

feed_forward(self, X)方法：     
用于进行前向传播计算。它接受一个输入数据X，并返回每一层的输出结果。  

backpropagation(self, X, y)方法:  
用于进行反向传播计算。它接受输入数据X和目标输出y，并返回损失值。  

initialize(self, X)方法:  
用于初始化网络中每一层的参数。  

train(self, X_train, X_test, y_train, y_test, batch_size, max_epochs, optimizer)方法:  
用于训练神经网络。它接受训练集和测试集的输入数据X_train和X_test，以及训练集和测试集的目标输出y_train和y_test。同时，它还接受批处理大小batch_size、最大迭代次数max_epochs和优化器optimizer作为参数。  

accuracy(self, y_predict, y_test)方法：  
用于计算预测结果的准确率。  
 
predict(self, X_predict)方法:  
用于对输入数据进行预测。它接受输入数据X_predict，并返回预测结果。  

save(self, file_name)方法:  
用于将训练好的模型保存到磁盘。  

read(self, file_name)方法：  
用于从磁盘中读取训练好的模型。  

In [11]:

class NeuralNetwork:
    def __init__(self):
        self._layers = []
        self._mses = []  # Mean Square Errors on training set
        self._ces = []  # Cross Entropy on training set
        self._accuracy = []  # Accuracy on test set
    def add_layer(self, layer):
        self._layers.append(layer)
    def feed_forward(self, X):
        """ Forward propagation """
        assert X.ndim == 2
        X = np.expand_dims(X, axis=0)  # X.shape = (1, h ,w)
        output = []
        for i in range(len(self._layers)):
            layer = self._layers[i]
            X = layer.activate(X)
            output.append(X)
            if __debug__:
                if np.max(np.abs(X)) > 2e2:
                    #pdb.set_trace()
                    print('Exploding Output at layer %s' %(i))  # lr or gradients too high
        return output  # outputs of each layer
    def backpropagation(self, X, y):
        output = self.feed_forward(X)
        loss = output[-1] - y  # gradients of categorical cross-entropy loss with softmax classification
        error = loss
        for i in reversed(range(len(self._layers))):  # propagate the error backward
            layer = self._layers[i]
            X_input = np.expand_dims(X, axis=0) if i == 0 else output[i - 1]
            error = layer.back_propagation(X_input, error, output[i])
            if __debug__:
                if np.max(np.abs(error)) > 1e1:
                    #pdb.set_trace()
                    print('Exploding Loss at layer %s' %(i))  # lr or gradients too high
        return loss
    def initialize(self, X):
        """ initialize parameters of layers """
        X = np.expand_dims(X, axis=0)
        for layer in self._layers:
            if type(layer) == ConvLayer:
                layer.initialize()
            if type(layer) == DenseLayer:
                layer.initialize(len(X))
            X = layer.activate(X)
    def train(self, X_train, X_test, y_train, y_test, batch_size, max_epochs, optimizer):
        assert X_train.ndim == 3  # X_train.shape = (n_image, h ,w)
        self.initialize(X_train[0])
        optimizer.initialize(self._layers, batch_size)
        y_onehot = np.zeros((y_train.shape[0], 10))  # one-hot coding
        y_onehot[np.arange(y_train.shape[0]), y_train] = 1
        mse_batch = 0.0
        for i in range(max_epochs):
            pbar = tqdm(total=len(X_train), desc='Batch MSE: %.4f'%(mse_batch), bar_format='{l_bar}{bar:40}| {n_fmt}/{total_fmt} {elapsed}')
            for j in range(len(X_train)):
                loss = self.backpropagation(X_train[j], y_onehot[j])
                mse_batch += np.mean(np.square(loss))
                if j % batch_size == batch_size - 1:
                    pbar.set_description('Batch MSE: %.4f'%(mse_batch / batch_size))
                    pbar.update(batch_size)
                    mse_batch = 0.0
                    optimizer.apply(self._layers)
            pbar.close()
            y_predict = self.predict(X_train)
            mse = np.mean(np.square(y_predict - y_onehot))
            self._mses.append(mse)
            ce = np.mean(-np.sum(y_onehot * np.log(y_predict), axis = 1))
            self._ces.append(ce)
            accuracy = self.accuracy(self.predict(X_test), y_test.flatten())
            self._accuracy.append(accuracy)
            print('Epoch: #%s, MSE: %.6f, CE: %.6f, Accuracy: %.2f%%' %(i+1, mse, ce, accuracy * 100))
            self.save('model.pkl')  # saving checkpoint
    def accuracy(self, y_predict, y_test):
        y_predict = np.argmax(y_predict, axis=1)
        return np.sum(y_predict == y_test) / len(y_test)
    def predict(self, X_predict):
        if X_predict.ndim == 2:
            X_predict = np.expand_dims(X_predict, axis=0)
        y_predict = np.zeros((X_predict.shape[0], 10))
        for i in range(len(X_predict)):
            X = np.expand_dims(X_predict[i], axis=0)
            for layer in self._layers:
                if type(layer) == DropoutLayer:  # skip dropout layer
                    continue
                X = layer.activate(X)
            y_predict[i] = X
        return y_predict
    def save(self, file_name):  # saving trained model to disk
        f = open(file_name, 'wb')
        pickle.dump(self, f)
        f.close()
    def read(self, file_name):  # reading trained model from disk
        try:
            print('Reading trained model from %s' %(file_name))
            f= open(file_name, 'rb')
            tmp_dict = pickle.load(f)
            f.close()
            for i in range(len(self.__dict__['_layers'])):
                if type(self._layers[i]) != type(tmp_dict._layers[i]):
                    print('Different network found in %s' %(file_name))
                    return
                elif type(self._layers[i]) == ConvLayer or type(self._layers[i]) == DenseLayer:
                    self._layers[i].weights = tmp_dict._layers[i].weights
                    self._layers[i].bias = tmp_dict._layers[i].bias
        except IOError: print('File %s is not accessible' %(file_name))

## 2 训练前的准备

### 2.1载入数据集
这段代码是用来加载MNIST数据集的。  
使用np.load()函数加载名为mnist.npz的文件，将其中的数据存储到相应的变量中。注意看，这个文件名叫小mnist.npz，这个文件名字不能更改，一旦更改，不堪设想，并且需要放在在同一个目录级别里。   
x_train和y_train：训练集的图片和标签，x_test和y_test：测试集的图片 标签。

如果在加载数据集的过程中出现了IOError异常，代码会打印出"mnist.npz is not accessible"的错误信息，表示无法访问mnist.  
这段代码的目的是加载MNIST数据集，以便后续的机器学习或深度学习任务中使用。  

In [12]:

try:
    f = np.load('mnist.npz')
    X_train, y_train = f['x_train'], f['y_train']
    X_test, y_test = f['x_test'], f['y_test'] 
    f.close()
except IOError: print('mnist.npz is not accessible')

### 2.2数据集的预处理
这段代码是将训练数据集和测试数据集中的像素值从整数类型转换为浮点类型，并进行归一化处理。  
它将像素值除以255，将像素值范围从0到255缩放到0到1之间，可以帮助提高模型的性能和收敛速度。

优点：  

加速模型收敛：将像素值归一化到0到1之间可以使数据的范围更小，有助于加快模型的收敛速度。这是因为在较小的数值范围内，模型的参数更新更加稳定。  
防止梯度消失或爆炸：在深度神经网络中，使用较大的像素值（例如0到255）可能导致梯度消失或爆炸的问题。通过将像素值缩放到0到1之间，可以减少这些问题的发生。  
平衡特征权重：归一化可以确保不同特征的权重在训练过程中更平衡。如果某个特征的值范围远大于其他特征，它可能会在模型训练中占据主导地位，而忽略其他特征的贡献。  
提高模型泛化能力：归一化可以帮助模型更好地泛化到新的未见过的数据。因为测试数据也要进行相同的归一化处理，所以模型在处理测试数据时会更加准确。  





In [13]:
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255


## 3.构建神经网络
这一块代码是构建了卷积神经网络（Convolutional Neural Network，CNN）模型，并加载预训练的模型参数。

ConvLayer：  
卷积层，通过应用一组滤波器（filters）对输入图像进行卷积操作，提取图像的特征。这里设置了8个滤波器，每个滤波器的大小为5x5，激活函数为ReLU。   

MaxPoolingLayer：  
最大池化层，通过在输入图像上滑动一个固定大小的窗口，并选择窗口内的最大值作为输出，从而减小特征图的尺寸。这里设置了2x2的池化窗口。  

ConvLayer：    
再次使用卷积层，这次设置了64个滤波器，每个滤波器的大小为3x3，激活函数为ReLU。 

MaxPoolingLayer：    
再次使用最大池化层，同样使用2x2的池化窗口。   

FlattenLayer：    
将特征图展平为一维向量，以便输入到全连接层。  

DenseLayer：  
全连接层，将展平后的特征向量映射到一个具有800个神经元的隐藏层，并使用Sigmoid激活函数。 

DropoutLayer：  
随机失活层，将隐藏层中的部分神经元以给定的概率（40%）随机关闭，以防止过拟合。  

DenseLayer：  
输出层，将隐藏层的输出映射到一个具有10个神经元的输出层，并使用Softmax激活函数，用于多分类问题。  

nn.read('model.pkl')：  
加载预训练的模型参数，如果存在名为'model.pkl'的文件。
  

这段代码构建了一个简单的CNN模型，用于图像分类任务。通过使用卷积和池化层，CNN可以自动学习图像的特征，并通过全连接层将学习到的特征映射到输出类别。加载预训练的模型参数可以方便地恢复之前训练好的模型状态，继续进行训练或进行预测。  

In [14]:
nn = NeuralNetwork()
nn.add_layer(ConvLayer(n_filters=8, kernel_size=(5, 5), activation='relu'))  # 1 image (28,28) => 8 image (24,24)
nn.add_layer(MaxPoolingLayer(pool_size=(2, 2)))  # 8 image (24,24) =>  8 image (12,12)
nn.add_layer(ConvLayer(n_filters=8, kernel_size=(3, 3), activation='relu'))  # 8 image (12,12) => 64 image (10,10)
nn.add_layer(MaxPoolingLayer(pool_size=(2, 2)))  # 64 image (10,10) =>  64 image (5,5)
nn.add_layer(FlattenLayer()) # 64 image (5,5) => 1600 pixels
nn.add_layer(DenseLayer(n_output=800, activation='sigmoid'))  # 1600 => 100
nn.add_layer(DropoutLayer(keep_prob=0.08))  # randomly shut down 40% neurons
nn.add_layer(DenseLayer(n_output=10, activation='softmax'))  # output layer, 100 => 10
nn.read('model.pkl')  # reading checkpoints if model.pkl exists

Reading trained model from model.pkl


## 4开始训练
time：  
用于计算代码执行时间。   


设置起始时间：    
time_start = time.time()：记录代码开始执行的时间。    

设置优化器和学习率：  
opt = Optimizers(optimizer='sgd', learning_rate= $a float number)：    
创建一个优化器对象，使用SGD或者adam作为优化器，并将学习率设置为一个数值。  

计算并打印代码执行时间：   
time_end = time.time()：记录代码执行结束的时间。    

print(f"Times Used: {time_end - time_start:.6f} seconds")：  
计算代码执行时间并打印出来。  

这段代码主要用于训练神经网络并可视化训练过程中的准确率变化。会打印整个代码执行的时间。  

In [ ]:
time_start = time.time()
opt = Optimizers(optimizer='sgd', learning_rate=0.007)
nn.train(X_train, X_test, y_train, y_test, batch_size=40, max_epochs=100, optimizer=opt)
time_end = time.time()
print("Times Used %.6f S"%(time_end - time_start))

## 5 探究：卷积层大小，池化层大小，随机失活率，学习率，优化器的选择（adam和SGD），batch size对CNN识别MNIST准确率和迭代时间的影响。
当探究卷积神经网络（CNN）的不同参数对于MNIST数据集的准确率和迭代时间的影响时，我们可以采用以下方法进行说明：

卷积层大小：
尝试不同的卷积层大小（例如3x3、5x5、7x7等）
增大卷积层大小可以增加模型的感受野，有助于提取更多的特征信息。
如果卷积层太小，可能无法捕捉到数据中的重要特征。
较大的卷积层可能会增加模型的参数量和计算复杂度，导致训练时间增加。

池化层大小：
尝试不同的池化层大小（例如2x2、3x3等）
增大池化层大小可以降低特征图的空间维度，减少计算量。
较大的池化层可能会导致信息丢失，影响模型的性能。
较小的池化层可能会保留更多的细节信息，但也会增加计算负担。

随机失活率：
尝试不同的随机失活率（例如0.1、0.3、0.5等）
适当的随机失活率可以帮助防止过拟合，提高模型的泛化能力。
较高的随机失活率可能导致模型欠拟合，减少模型的性能。
较低的随机失活率可能无法有效地减少过拟合。

学习率：
尝试不同的学习率（例如0.001、0.01、0.1等）
适当的学习率可以加快模型的收敛速度。
较高的学习率可能导致模型发散，训练过程不稳定。
较低的学习率可能导致模型收敛速度过慢，需要更多的训练时间。

优化器的选择（Adam和SGD）：

使用Adam和SGD两种优化器分别进行训练
Adam优化器通常具有较快的收敛速度和较好的泛化能力。
SGD优化器通常需要较长的训练时间，但在一些情况下可能达到更好的性能。
具体情况具体分析。

批次大小：
尝试不同的批次大小（例如32、64、128等）
较大的批次大小可以加快训练速度，但也可能导致内存不足或计算资源不足。
较小的批次大小可以提高模型的泛化能力，但训练过程可能更加不稳定。


## 6 结论：如何对最优化参数进行选择?
池化层大小：池化层通常用于减小特征图的空间尺寸，并保留主要特征。常见的池化操作有最大池化（Max Pooling）和平均池化（Average Pooling）。  
1）对于MNIST数据集，通常使用2x2的最大池化层效果较好。  
  
随机失活率：随机失活（Dropout）是一种正则化技术，可以减少过拟合问题。通过在训练过程中随机将一部分神经元的输出设为0，可以强迫网络去学习更加鲁棒的特征  
2）随机失活率在0.2到0.5之间效果较好。    
  
学习率：学习率决定了网络参数的更新速度，过小的学习率会导致收敛缓慢，而过大的学习率则可能导致无法收敛。  
3）从0.001开始尝试，根据实验结果调整。本数据集设置为0.007  

优化器的选择：优化器决定了如何更新网络参数。常见的优化器有随机梯度下降（SGD）和自适应矩估计（Adam）。SGD在一些情况下可能会陷入局部最优，而Adam通常能更快地收敛。  
4）因此，对于MNIST数据集，建议使用Adam优化器。（本数据集用的SGD）  
  
Batch Size：Batch Size指的是每次迭代训练时使用的样本数量。较大的Batch Size可以加快训练速度，但可能会占用更多的内存。对于MNIST数据  集，  
5）可以尝试使用32或64的Batch Size。本数据集合选的40  





### 6.1 个人最优参数设置
个人使用随机梯度下降（SGD作为优化算法，学习率为0.007。批处理大小为40，过滤器的大小为5x5，最大池化层2x2，随机失活层:0.08的概率随机关闭40%的神经元,训练100次，可以得到如下部分训练结果，每轮训练时间为12分钟，准确率在98.8%左右，100次花了近1天时间。

Reading trained model from model.pkl   
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:49   
Epoch: #1, MSE: 0.000336, CE: 0.008606, Accuracy: 98.79%   
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:41   
Epoch: #2, MSE: 0.000322, CE: 0.008199, Accuracy: 98.82%   
Batch MSE: 0.0002: 100%|████████████████████████████████████████| 60000/60000 12:39   
Epoch: #3, MSE: 0.000328, CE: 0.008238, Accuracy: 98.80%     
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:41   
Epoch: #4, MSE: 0.000241, CE: 0.006719, Accuracy: 98.86%   
Batch MSE: 0.0009: 100%|████████████████████████████████████████| 60000/60000 12:39  
Epoch: #5, MSE: 0.000287, CE: 0.007235, Accuracy: 98.89%    
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:40    
Epoch: #6, MSE: 0.000239, CE: 0.006726, Accuracy: 98.85%      
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:41   
Epoch: #7, MSE: 0.000249, CE: 0.006764, Accuracy: 98.90%     
Batch MSE: 0.0006: 100%|████████████████████████████████████████| 60000/60000 12:40    
Epoch: #8, MSE: 0.000254, CE: 0.006742, Accuracy: 98.83%     
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:40    
Epoch: #9, MSE: 0.000189, CE: 0.005620, Accuracy: 98.89%    
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:40    
Epoch: #10, MSE: 0.000181, CE: 0.005352, Accuracy: 98.92%  
........................    
Epoch: #90, MSE: 0.000072, CE: 0.002913, Accuracy: 98.90%     
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36  
Epoch: #91, MSE: 0.000072, CE: 0.002914, Accuracy: 98.92%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36   
Epoch: #92, MSE: 0.000072, CE: 0.002915, Accuracy: 98.93%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36    
Epoch: #93, MSE: 0.000072, CE: 0.002915, Accuracy: 98.92%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:37  
Epoch: #94, MSE: 0.000072, CE: 0.002915, Accuracy: 98.92%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:38   
Epoch: #95, MSE: 0.000072, CE: 0.002915, Accuracy: 98.93%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36    
Epoch: #96, MSE: 0.000072, CE: 0.002915, Accuracy: 98.91%   
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36   
Epoch: #97, MSE: 0.000072, CE: 0.002913, Accuracy: 98.92%   
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:36  
Epoch: #98, MSE: 0.000072, CE: 0.002915, Accuracy: 98.93%  
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:37   
Epoch: #99, MSE: 0.000072, CE: 0.002914, Accuracy: 98.91%   
Batch MSE: 0.0000: 100%|████████████████████████████████████████| 60000/60000 12:37   
Epoch: #100, MSE: 0.000072, CE: 0.002913, Accuracy: 98.92%   
Times Used 75731.191981 S   

## 7.其他说明

### 7.1 错误识别预览

In [1]:
def plot_misprediction(test_X, output_label): #显示被预测数字图片
    image = np.array(test_X)
    image = image.reshape(28, 28)
    fig = plt.figure('testdata\'s label')
    plotwindow = fig.add_subplot(111)
    plt.axis('off')
    String_0 = "The true number is "+str(true_label)
    String_1 = String_0+", and the predicted number is "+str(output_label)
    plt.title(String_1)
    plt.imshow(image, cmap='gray')
    plt.show()
    plt.close()
    return 0

### 7.2 测试
随机选择数据进行预测，并显示预测结果的图像。

In [ ]:


rand_x = 0
while rand_x == 0:   # 随机选择数据进行预测
    i = int(input("input the number of picture to test: "))  # 测试值i的范围在0~10000之内
    if i == -1: # 当输入i=-1，停止预测
        mutex = 1
        print("Test Over！")
        break
    test_X = X_test[i, :]
    true_label = y_test[i]
    output_label = np.argmax(nn.predict(test_X), axis = 1)
    print(" No.",i," picture's number is:", true_label, end=',')
    print(" the predicted number is:", output_label)
    plot_misprediction(test_X, output_label)

## 8.致谢
在完成本作业的过程中，我学到了很多之前没能学会的东西，我受益于许多人的帮助和支持，在此向他们表示感谢。首先，我要感谢Hong-qian SANG,月也是本门课的lecturer，他带领我第一次体验到这种项目主体的讨论式的课堂，并在课下帮我解决技术性问题。其次我要感谢Xiao-long ZHU，他在我的作业完成里头和研究过程中给予了我很多的资源。其次，我要感谢zi-hao HUANG，zhou-tong ZHANG，yan-ci ZHOU，他们在我学习和研究中给予了我很多的鼓励和支持，让我能够一直保持着学习的热情和动力。最后，我要感谢所有为深度学习和计算机视觉技术做出贡献的学者和工程师，我使用了Chat GPT 3.5 Turbo帮我修改完善代码，学习了Andrew Ng的机器学习课程，阅读了Kunihiko Fukushima,LeCun, Y等人的论文，查看了Yong-le LI，3Blue1Brown等人的CNN视频，他们的研究和努力让我能够站在巨人的肩膀上，不断探索和创新。

## 9参考文献与引用列表
[1]But what is a convolution? (2022) YouTube. Available at: https://www.youtube.com/watch?v=KuXjwB4LzSA (Accessed: 29 June 2023).   

[2]But what is a neural network? (no date) 3Blue1Brown. Available at: https://www.3blue1brown.com/lessons/neural-networks (Accessed: 29 June 2023).   

[3]Carter, D.S. and S. (no date) Tensorflow - Neural Network Playground, A Neural Network Playground. Available at: https://playground.tensorflow.org/ (Accessed: 29 June 2023).   

[4]Fukushima, K. (1980) ‘Neocognitron: A self-organizing neural network model for a mechanism of pattern recognition unaffected by shift in position’, Biological Cybernetics, 36(4), pp. 193–202. doi:10.1007/bf00344251.   

[5]Krizhevsky, A., Sutskever, I. and Hinton, G.E. (2017) ‘ImageNet classification with deep convolutional Neural Networks’, Communications of the ACM, 60(6), pp. 84–90. doi:10.1145/3065386.   

[6]Lecun, Y. et al. (1998) ‘Gradient-based learning applied to document recognition’, Proceedings of the IEEE, 86(11), pp. 2278–2324. doi:10.1109/5.726791.   

[7]LeCun, Y., Bengio, Y. and Hinton, G. (2015) ‘Deep learning’, Nature, 521(7553), pp. 436–444. doi:10.1038/nature14539.   

[8]Machine learning yearning (no date) dBooks.org. Available at: https://www.dbooks.org/machine-learning-yearning-1501/ (Accessed: 29 June 2023).   

[9]人工智能（一）机器学习和神经网络 (no date) 哔哩哔哩. Available at: https://www.bilibili.com/read/cv10327726/ (Accessed: 29 June 2023).   

[10]神思科学 deepthought science (no date) 神思科学 DeepThought Science. 
Available at: https://www.dtscience.com.cn/ (Accessed: 29 June 2023).   
